### Numerical differentiation and integration

#### Take-aways

After studying this chapter, we will be able to

TBF


#### Overview

##### Settings/Notation

Common settings

| symbol | setting |
|---|---|
| $m$ | a positive integer; number of observations; number of equations |
| $n$ | a positive integer; number of parameters; number of unknowns |
| $A$ | full rank $m$-by-$n$ matrix with $m \gg n$ (skinny matrix) |
| $b$ | (column) vector of length $m$ |
| $x$ | (column) vector of length $n$ |


##### Problem of interest

Given an over-determined (or inconsistent) system $``Ax=b"$, find the *best* $x$, that is the *residual* $ Ax - b $ is the smallest in some sense.


##### Methods 

1. Least square

##### Important concepts


##### Important preliminary

- Projections (Linear algebra)
- Gradient of vector field (Vector calculus 1)

##### Summary

- (1st index matching requirement on $A$ and $b$) The number of rows of $A$ and the lenth of $b$ must match. 
- The number of columns of $A$ must match the length of the solution $x$.


##### Numerical example

In [8]:
import numpy as np

# 1. Create A and b for a least square problem
A = np.arange(3*5, dtype=np.float64).reshape((3,5))**2
b = np.arange(5, dtype=np.float64).reshape((5,1))**2

# 2. Solve the least square problem using pseudo inverse
x = np.linalg.pinv(A.T)@b

# 3. Check the result
print(A.T)
print(b)
print(x)

# 4. Check the residual
res = A.T@x - b
print(res)

# 5. Check the residual for the true solution (cheating)
x_ = np.array([1,0,0]).reshape((3,1))
print(A.T@x_ - b)

[[  0.  25. 100.]
 [  1.  36. 121.]
 [  4.  49. 144.]
 [  9.  64. 169.]
 [ 16.  81. 196.]]
[[ 0.]
 [ 1.]
 [ 4.]
 [ 9.]
 [16.]]
[[1.00000000e+00]
 [0.00000000e+00]
 [1.66533454e-16]]
[[1.66533454e-14]
 [2.15383267e-14]
 [2.93098879e-14]
 [4.08562073e-14]
 [5.32907052e-14]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]


> **Question**
>
> Remove squaring (`**2`) in the above and run the code. What happens? Can you explain why that happens? What does this example remind us of about the assumptions of the least square problem?
>
> (Reminder) This is **about atmosphere**, not getting it right.
> 
> 1. Think for a short time.
> 2. Share your guess with your pair.
> 3. Type your answer in clicker.
> 4. Feel free to say out loud.

##### Analysis


> **Question**
>
> TBF
>
> (Reminder) This is **about atmosphere**, not getting it right.
> 
> 1. Think for a short time.
> 2. Share your guess with your pair.
> 3. Type your answer in clicker.
> 4. Feel free to say out loud.

#### Method1 (TBC)

##### Method


##### Summary


##### Analysis


> **Question**
>
> TBF
>
> (Reminder) This is **about atmosphere**, not getting it right.
> 
> 1. Think for a short time.
> 2. Share your guess with your pair.
> 3. Type your answer in clicker.
> 4. Feel free to say out loud.

#### Method1 (TBC)

##### Method


##### Summary


##### Analysis


> **Question**
>
> TBF
>
> (Reminder) This is **about atmosphere**, not getting it right.
> 
> 1. Think for a short time.
> 2. Share your guess with your pair.
> 3. Type your answer in clicker.
> 4. Feel free to say out loud.

---
This work is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)